In [1]:
import numpy as np
import pandas as pd


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
# IsAlone: 혼자임 - 1, 혼자가 아님 - 0
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [2]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [3]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [4]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


In [5]:
# from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)

# **random forest**

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [7]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [8]:
sub = pd.read_csv('gender_submission.csv')

In [9]:
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission_randomforest.csv', index=False)

# LightGBM

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                 random_state=0, stratify=y_train)

In [11]:
categorical_fetures = ['Embarked', 'Pclass', 'Sex']

In [12]:
# pip install --upgrade pip

In [13]:
!pip install lightgbm
# !pip install --upgrade lightgbm
# !pip show lightgbm

In [14]:
import lightgbm

OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/usr/lib/libomp.dylib' (no such file), '/opt/homebrew/Cellar/libomp/14.0.0/lib/libomp.dylib' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
from lightgbm import LGBMClassifier

lgb_train = lgb.Dataset(X_train, y_train,
                       categorical_feture=categorical_fetures)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                       categorical_feture=categorical_fetures)

params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                 valid_sets=[lgb_train, lgb_eval],
                 verbose_eval=10,
                 num_boost_round=1000,
                 early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
pip install lxml